In [1]:
!cp /kaggle/input/gdcm-conda-install/gdcm.tar .
!tar -xvzf gdcm.tar
!conda install --offline ./gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2
!rm -rf ./gdcm.tar

gdcm/
gdcm/conda-4.8.4-py37hc8dfbb8_2.tar.bz2
gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2
gdcm/libjpeg-turbo-2.0.3-h516909a_1.tar.bz2

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | / done
Executing transaction: \ done


In [2]:
## MMDetection compatible torch installation
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torch-1.7.0+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchvision-0.8.1+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchaudio-0.7.0-cp37-cp37m-linux_x86_64.whl' --no-deps

## Compatible Cuda Toolkit installation
!mkdir -p /kaggle/tmp && cp /kaggle/input/pytorch-170-cuda-toolkit-110221/cudatoolkit-11.0.221-h6bb024c_0 /kaggle/tmp/cudatoolkit-11.0.221-h6bb024c_0.tar.bz2 && conda install /kaggle/tmp/cudatoolkit-11.0.221-h6bb024c_0.tar.bz2 -y --offline

## MMDetection Offline Installation
!pip install '/kaggle/input/mmdetectionv2140/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminal-0.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminaltables-3.1.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/mmcv_full-1_3_8-cu110-torch1_7_0/mmcv_full-1.3.8-cp37-cp37m-manylinux1_x86_64.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/pycocotools-2.0.2/pycocotools-2.0.2' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/mmpycocotools-12.0.3/mmpycocotools-12.0.3' --no-deps

!cp -r /kaggle/input/mmdetectionv2140/mmdetection-2.14.0 /kaggle/working/
!mv /kaggle/working/mmdetection-2.14.0 /kaggle/working/mmdetection
%cd /kaggle/working/mmdetection
!pip install -e . --no-deps
%cd /kaggle/working/

Processing /kaggle/input/pytorch-170-cuda-toolkit-110221/torch-1.7.0+cu110-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.7.0
    Uninstalling torch-1.7.0:
      Successfully uninstalled torch-1.7.0
Processing /kaggle/input/pytorch-170-cuda-toolkit-110221/torchvision-0.8.1+cu110-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.8.1
    Uninstalling torchvision-0.8.1:
      Successfully uninstalled torchvision-0.8.1
Processing /kaggle/input/pytorch-170-cuda-toolkit-110221/torchaudio-0.7.0-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.7.0a0+ac17b64
    Uninstalling torchaudio-0.7.0a0+ac17b64:
      Successfully uninstalled torchaudio-0.7.0a0+ac17b64

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction:

In [3]:
import sys
sys.path.append('/kaggle/input/efficientnet-keras-dataset/efficientnet_kaggle')
sys.path.append('/kaggle/input/weightedboxesfusionofficial/ensemble_boxes')
import efficientnet.tfkeras as efn
from ensemble_boxes_wbf import weighted_boxes_fusion

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from glob import glob
import gc
from tqdm import tqdm
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tensorflow.keras import Model, Sequential
import cv2
import math
from PIL import Image
from shutil import copytree
import os
from tensorflow.keras import layers

In [5]:
df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')

In [6]:
fast_sub = False
if df.shape[0] == 2477:
    fast_sub = True
else:
    fast_sub = False

In [7]:
fast_sub

True

In [8]:
filepaths = glob('/kaggle/input/siim-covid19-detection/test/**/*dcm',recursive=True)
if fast_sub:
    filepaths = filepaths[:5]
test_df = pd.DataFrame(data={'filepath': filepaths})
test_df['image_id'] = test_df.filepath.map(lambda x: x.split('/')[-1].replace('.dcm', '')+'_image')
test_df['study_id'] = test_df.filepath.map(lambda x: x.split('/')[-3].replace('.dcm', '')+'_study')
test_df

,filepath,image_id,study_id
0,/kaggle/input/siim-covid19-detection/test/2fb1...,a29c5a68b07b_image,2fb11712bc93_study
1,/kaggle/input/siim-covid19-detection/test/19c6...,9850b5470fd6_image,19c66935e737_study
2,/kaggle/input/siim-covid19-detection/test/2fc5...,8d6dea06a032_image,2fc50bf199cd_study
3,/kaggle/input/siim-covid19-detection/test/97c5...,dfc5c09a50bc_image,97c5d6eb413d_study
4,/kaggle/input/siim-covid19-detection/test/593c...,7230234e120a_image,593c3f815635_study


In [9]:
def image_process(image_array, image_size=(600, 600)):
    image = tf.cast(image_array, tf.float32) / 255.
    image = tf.image.resize(image, image_size)
    return image

In [10]:
def apply_augmentation(image):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image

In [11]:
def dicom2array(path, voi_lut=True, fix_monochrome=True, yolo=False):
    dicom = pydicom.read_file(path)
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    if yolo:
        return data
    image = cv2.cvtColor(data, cv2.COLOR_GRAY2RGB)
    return image

In [12]:
BATCH_SIZE = 5 if fast_sub else 32

In [13]:
class TestDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_paths, TTA=None, batch_size=BATCH_SIZE):
        self.image_paths = image_paths
        self.batch_size = BATCH_SIZE
        self.TTA = TTA
        
    def __len__(self):
        return math.ceil(len(self.image_paths) / self.batch_size)
    
    def __getitem__(self, indx):
        images = self.image_paths[indx*self.batch_size:(indx+1)*self.batch_size]
        images_batch = np.array(list(map(dicom2array, images)))
        images_batch = np.array(list(map(image_process, images_batch)))
        if self.TTA:
            images_batch = np.array(list(map(apply_augmentation, images_batch)))
        return images_batch

In [14]:
model = tf.keras.Sequential([
    efn.EfficientNetB7(
        input_shape=(600, 600, 3),
        weights='imagenet',
        include_top=False),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [15]:
test_dataset = TestDataGenerator(filepaths)

In [16]:
#model = build_model()

In [17]:
num_folds = 5
predict = []
for i in tqdm(range(num_folds)):
    model_path = f'../input/slim-weights-effnet/model{i}.h5'
    model.load_weights(model_path)
    pred = model.predict(test_dataset)
    predict.append(pred)
    del pred 
    gc.collect()
predictions = np.mean(predict, axis=0)

  0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':
100%|██████████| 5/5 [00:59<00:00, 11.89s/it]


In [18]:
del predict, model
gc.collect()

76587

In [19]:
#test_dataset = TestDataGenerator(test_df.filepath.tolist())

In [20]:
#final_pred = np.mean([predictions, mt_predictions], axis=0)

In [21]:
#mt_predictions

In [22]:
data = {'id': test_df.study_id.tolist()}
study_df = pd.DataFrame(data=data)
study_df['PredictionString'] = None
for i in tqdm(range(predictions.shape[0])):
    study_df.loc[i, 'PredictionString'] = f'negative {predictions[i][0]:.3f} 0 0 1 1 \
    typical {predictions[i][1]:.3f} 0 0 1 1 \
    indeterminate {predictions[i][2]:.3f} 0 0 1 1 \
    atypical {predictions[i][3]:.3f} 0 0 1 1'

study_df.sort_values(by=['id'])
#study_df = study_df.drop_duplicates(subset=['id'])
#image_df = pd.DataFrame({'id':test_df.image_id.tolist(),
                        # 'PredictionString':["none 1 0 0 1 1"]*len(test_df.image_id.tolist())})
#study_df = study_df.drop_duplicates(subset=['id'])
#final_df = pd.concat([study_df.sort_values(by=['id']), image_df])
#final_df.to_csv('submission.csv', index=False)

100%|██████████| 5/5 [00:00<00:00, 1292.54it/s]


,id,PredictionString
1,19c66935e737_study,negative 0.025 0 0 1 1 typical 0.748 0 0 1...
0,2fb11712bc93_study,negative 0.002 0 0 1 1 typical 0.948 0 0 1...
2,2fc50bf199cd_study,negative 0.020 0 0 1 1 typical 0.810 0 0 1...
4,593c3f815635_study,negative 0.001 0 0 1 1 typical 0.710 0 0 1...
3,97c5d6eb413d_study,negative 0.096 0 0 1 1 typical 0.608 0 0 1...


In [23]:
study_df = study_df.drop_duplicates(subset=['id'])
study_df.shape

(5, 2)

In [24]:
#import tensorflow as tf
#import efficientnet.tfkeras as efn

MODEL_PATH = '/kaggle/input/siimcovid19efnb7trainfold052class'

#image_df['none'] = 0
#label_cols = ['none']
model = tf.keras.Sequential([
            efn.EfficientNetB7(
                input_shape=(512, 512, 3),
                weights='imagenet',
                include_top=False),
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

num_folds = 5
predict = []
for i in tqdm(range(num_folds)):
    model_path = f'{MODEL_PATH}/model{i}.h5'
    model.load_weights(model_path)
    pred = model.predict(test_dataset)
    predict.append(pred)
    del pred 
    gc.collect()
    
predictions = np.mean(predict, axis=0)

sub_df = pd.DataFrame(data={'id': test_df['image_id'].tolist()})
sub_df['none'] = predictions #sum([model.predict(test_dataset, verbose=1) for model in models]) / len(models)

del model
#del models0, models1, models2, models3, models4
del test_dataset, predictions
gc.collect()

  0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':
100%|██████████| 5/5 [00:58<00:00, 11.71s/it]


76567

In [25]:
sub_df.head()

,id,none
0,a29c5a68b07b_image,0.058766
1,9850b5470fd6_image,0.096071
2,8d6dea06a032_image,0.133567
3,dfc5c09a50bc_image,0.356122
4,7230234e120a_image,0.062900


In [26]:
#del test_dataset, predict, model
#gc.collect()

In [27]:
############# cascade_rnn part ##############

In [28]:
from numba import cuda
import torch
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

<weakproxy at 0x7ff6f6b7c650 to Device at 0x7ff7882722d0>

In [29]:
import sys
sys.path.append('/kaggle/working/mmdetection')
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

from mmdet.apis import set_random_seed

import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector

import mmcv
from mmcv import Config
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmdet.apis import single_gpu_test
from mmdet.datasets import build_dataloader, build_dataset

11.0
GCC 7.3


In [30]:
baseline_cfg_path = '../input/cascade-rnn/job4_cascade_rcnn_x101_32x4d_fpn_1x_coco.py'
cfg = Config.fromfile(baseline_cfg_path)

In [31]:
cfg.classes = ("Covid_Abnormality")
cfg.data.test.img_prefix = ''
cfg.data.test.classes = cfg.classes

for head in cfg.model.roi_head.bbox_head:
    head.num_classes = 1

cfg.seed = 211
set_random_seed(211, deterministic=False)
cfg.gpu_ids = [0]

cfg.data.test.pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1333, 800),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip', direction='horizontal'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='DefaultFormatBundle'),
                    dict(type='Collect', keys=['img'])
                ])
        ]

cfg.test_pipeline = [
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1333, 800),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip', direction='horizontal'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='DefaultFormatBundle'),
                    dict(type='Collect', keys=['img'])
                ])
        ]


cfg.model.test_cfg.rcnn.score_thr = 0.001

In [32]:
WEIGHTS_FILE = '../input/siimmmdetectioncascadercnnweightbias/epoch_10.pth'

In [33]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device.type)
import torchvision
print(torch.__version__, torch.cuda.is_available())
print(get_compiling_cuda_version())
print(get_compiler_version())

cuda
1.7.0+cu110 True
11.0
GCC 7.3


In [34]:
#device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
options = dict(classes = ("Covid_Abnormality"))
model = init_detector(cfg, WEIGHTS_FILE, device='cuda:0')

/kaggle/working/mmdetection/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '


Use load_from_local loader


/kaggle/working/mmdetection/mmdet/apis/inference.py:47: UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn('Class names are not saved in the checkpoint\'s '


In [35]:
results = []
score_threshold = cfg.model.test_cfg.rcnn.score_thr

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
'''def format_pred(boxes, scores, labels):
    pred_strings = []
    label_str = ['opacity']
    for label, score, bbox in zip(labels, scores, boxes):
        xmin, ymin, xmax, ymax = bbox.astype(np.int64)
        pred_strings.append(f"{label_str[int(label)]} {score:.16f} {xmin} {ymin} {xmax} {ymax}")
    return " ".join(pred_strings)'''


'def format_pred(boxes, scores, labels):\n    pred_strings = []\n    label_str = [\'opacity\']\n    for label, score, bbox in zip(labels, scores, boxes):\n        xmin, ymin, xmax, ymax = bbox.astype(np.int64)\n        pred_strings.append(f"{label_str[int(label)]} {score:.16f} {xmin} {ymin} {xmax} {ymax}")\n    return " ".join(pred_strings)'

In [37]:
model.to(device)
model.eval()

CascadeRCNN(
  (backbone): ResNeXt(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): S

In [38]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    im = Image.fromarray(array)
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    return im

In [39]:
split = 'test'
save_dir = f'/kaggle/tmp/{split}/'
os.makedirs(save_dir, exist_ok=True)

In [40]:
W = []
H = []
for filepath in tqdm(filepaths):
    xray = dicom2array(filepath, yolo=True)
    W.append(xray.shape[0])
    H.append(xray.shape[1])
    im = resize(xray, size=512) 
    image = filepath.split('/')[-1].replace('.dcm', '') + '.png'
    im.save(os.path.join(save_dir, image))

100%|██████████| 5/5 [00:02<00:00,  2.04it/s]


In [41]:
copytree('/kaggle/input/yolov5-repo/yolov5-master', '/kaggle/tmp/yolov5')

'/kaggle/tmp/yolov5'

In [42]:
#! ls /kaggle/working/test/

In [43]:
#copytree('/kaggle/input/yolov5-repo/yolov5-master', '/kaggle/tmp/yolov5')
#os.chdir('/kaggle/tmp/yolov5')
#sys.path.append('/kaggle/input/yolov5-repo/yolov5-master')
#import detect

In [44]:
MODEL_PATH = '../input/weights-of-yolov5-150-epochs/best.pt'
TEST_PATH = '/kaggle/tmp/test'
IMG_SIZE = 512
CONF_SCORE = 0.001

In [45]:
data = {'filepath': filepaths, 'w': W, 'h': H}
meta = pd.DataFrame(data=data)
meta['id'] = meta.filepath.apply(lambda x: x.split('/')[-1].split('.')[0])

In [46]:
meta.head()

,filepath,w,h,id
0,/kaggle/input/siim-covid19-detection/test/2fb1...,2320,2828,a29c5a68b07b
1,/kaggle/input/siim-covid19-detection/test/19c6...,2330,2382,9850b5470fd6
2,/kaggle/input/siim-covid19-detection/test/2fc5...,2422,3344,8d6dea06a032
3,/kaggle/input/siim-covid19-detection/test/97c5...,1140,1387,dfc5c09a50bc
4,/kaggle/input/siim-covid19-detection/test/593c...,2318,2383,7230234e120a


In [47]:
!python /kaggle/tmp/yolov5/detect.py --weights {MODEL_PATH}\
--img 512\
--conf {CONF_SCORE}\
--iou 0.5\
--augment\
--max 50\
--source {TEST_PATH}\
--save-txt --save-conf --exist-ok

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


detect: weights=['../input/weights-of-yolov5-150-epochs/best.pt'], source=/kaggle/tmp/test, imgsz=512, conf_thres=0.001, iou_thres=0.5, max_det=50, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=True, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False
requirements: /kaggle/working/requirements.txt not found, check failed.
image 1/5 /kaggle/tmp/test/7230234e120a.png: 512x512 24 0. opacitys, Done. (0.104s)
image 2/5 /kaggle/tmp/test/8d6dea06a032.png: 512x512 34 0. opacitys, Done. (0.083s)
image 3/5 /kaggle/tmp/test/9850b5470fd6.png: 512x512 41 0. opacitys, Done. (0.083s)
image 4/5 /kaggle/tmp/test/a29c5a68b07b.png: 512x512 18 0. opacitys, Done. (0.082s)
image 5/5 /kaggle/tmp/test/dfc5c09a50bc.png: 512x512 35 0. opacitys, Done. (0.084s)
Results saved to runs/detect/exp
5 labels saved to runs/detect/exp/labels
Done. (0.

In [48]:
PRED_PATH = 'runs/detect/exp/labels/'
prediction_files = os.listdir(PRED_PATH)

In [49]:
#prediction_files

In [50]:
def yolo2voc(image_height, image_width, bboxes):

    bboxes = bboxes.copy().astype(float)

    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

    return bboxes

In [51]:
def get_conf_bboxes(file_path):
    confidence = []
    bboxes = []
    with open(file_path, 'r') as file:
        for line in file:
            preds = line.strip('\n').split(' ')
            preds = list(map(float, preds))
            confidence.append(preds[-1])
            bboxes.append(preds[1:-1])
    return confidence, bboxes

In [52]:
ids = []

fin_predict = [] 
with torch.no_grad(): 
    for file_path in tqdm(glob(PRED_PATH+'*.txt')): 
        ############### yolo part ################ 
        image_id = file_path.split('/')[-1].split('.')[0] 
        ids.append(image_id) 
        w, h = meta.loc[meta.id==image_id, ['w', 'h']].values[0] 
        yolo_conf, yolo_bboxes = get_conf_bboxes(file_path)

        ########### cascade_rnn part ############
        im_path = TEST_PATH + '/' + image_id + '.png'
        predictions = inference_detector(model, im_path)
        crnn_bboxes, crnn_conf = [], []

        for k, pred in enumerate(predictions):
            if pred.size != 0:
                if len(crnn_conf) == 0:
                    crnn_bboxes = np.array(pred[:, :4])
                    crnn_conf = np.array(pred[:, 4])

                else:
                    crnn_bboxes = np.concatenate((crnn_bboxes, np.array(pred[:, :4])))
                    crnn_conf = np.concatenate((crnn_bboxes, np.array(pred[:, 4])))

        indexes = np.where(crnn_conf > CONF_SCORE)
        
        crnn_bboxes = crnn_bboxes[indexes]
        crnn_conf = crnn_conf[indexes]

        yolo_classes = np.zeros(len(yolo_conf))
        crnn_classes = np.zeros(len(crnn_conf))
        
        # change yolo format to [x1, y1, x2, y2]
        yolo_bboxes = np.array(yolo_bboxes)
        yolo_bboxes = yolo2voc(h, w, yolo_bboxes)
        
        # scale cascade rnn pred bboxes
        h_ratio = h/512
        w_ratio = w/512
        
        crnn_bboxes[:, [0, 2]] *= w_ratio
        crnn_bboxes[:, [1, 3]] *= h_ratio
        
        SCALED_VALUE = 10000
        
        yolo_bboxes /= SCALED_VALUE
        crnn_bboxes /= SCALED_VALUE
        
        
        bboxes, confidences, labels = weighted_boxes_fusion(boxes_list=[yolo_bboxes, crnn_bboxes],
                                                            scores_list=[yolo_conf, crnn_conf],
                                                            labels_list=[yolo_classes, crnn_classes],
                                                            iou_thr = 0.6)
        #print(len(bboxes))
        #if len(bboxes) > 50:
            #bboxes = bboxes[:50]
            #confidences = confidences[:50]
            #labels = labels[:50]
        print(len(bboxes))
        bboxes = np.round(bboxes*SCALED_VALUE).astype(int)
        pred_string = ''
        for j, conf in enumerate(confidences):
            pred_string += f'opacity {conf} ' + ' '.join(map(str, bboxes[j])) + ' '
        fin_predict.append(pred_string[:-1])

  0%|          | 0/5 [00:00<?, ?it/s]/kaggle/working/mmdetection/mmdet/core/anchor/anchor_generator.py:323: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/kaggle/working/mmdetection/mmdet/core/anchor/anchor_generator.py:360: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  '``single_level_grid_anchors`` would be deprecated soon. '
 20%|██        | 1/5 [00:01<00:05,  1.25s/it]/kaggle/working/mmdetection/mmdet/core/anchor/anchor_generator.py:323: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/kaggle/working/mmdetection/mmdet/core/anchor/anchor_generator.py:360: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  '``single_level_grid_anchors`` would be deprecated soon. '


52
28


 80%|████████  | 4/5 [00:01<00:00,  3.35it/s]

62
38


100%|██████████| 5/5 [00:01<00:00,  2.70it/s]

53


In [53]:
#sub_df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [54]:
image_df = pd.DataFrame(data={'id': ids, 'PredictionString': fin_predict})
image_df['id'] = image_df.id.apply(lambda x: x + '_image')
image_df.head()

,id,PredictionString
0,dfc5c09a50bc_image,opacity 0.3846456706523895 716 412 928 935 opa...
1,a29c5a68b07b_image,opacity 0.6888105869293213 1547 825 2229 2318 ...
2,9850b5470fd6_image,opacity 0.4906483292579651 290 693 984 1933 op...
3,7230234e120a_image,opacity 0.571875274181366 1390 1190 2106 1946 ...
4,8d6dea06a032_image,opacity 0.453635036945343 1495 1168 1967 2396 ...


In [55]:
image_df = image_df.merge(sub_df, on='id', how='left')
image_df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,PredictionString,none
0,dfc5c09a50bc_image,opacity 0.3846456706523895 716 412 928 935 opa...,0.356122
1,a29c5a68b07b_image,opacity 0.6888105869293213 1547 825 2229 2318 ...,0.058766
2,9850b5470fd6_image,opacity 0.4906483292579651 290 693 984 1933 op...,0.096071
3,7230234e120a_image,opacity 0.571875274181366 1390 1190 2106 1946 ...,0.062900
4,8d6dea06a032_image,opacity 0.453635036945343 1495 1168 1967 2396 ...,0.133567


In [56]:
for i in range(len(image_df)):
    image_df.loc[i, 'PredictionString'] = image_df.loc[i, 'PredictionString'] + ' none ' + str(image_df.loc[i, 'none'])+ ' 0 0 1 1'

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [57]:
image_df = image_df[['id', 'PredictionString']]
image_df.head()

,id,PredictionString
0,dfc5c09a50bc_image,opacity 0.3846456706523895 716 412 928 935 opa...
1,a29c5a68b07b_image,opacity 0.6888105869293213 1547 825 2229 2318 ...
2,9850b5470fd6_image,opacity 0.4906483292579651 290 693 984 1933 op...
3,7230234e120a_image,opacity 0.571875274181366 1390 1190 2106 1946 ...
4,8d6dea06a032_image,opacity 0.453635036945343 1495 1168 1967 2396 ...


In [58]:
study_df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,PredictionString
0,2fb11712bc93_study,negative 0.002 0 0 1 1 typical 0.948 0 0 1...
1,19c66935e737_study,negative 0.025 0 0 1 1 typical 0.748 0 0 1...
2,2fc50bf199cd_study,negative 0.020 0 0 1 1 typical 0.810 0 0 1...
3,97c5d6eb413d_study,negative 0.096 0 0 1 1 typical 0.608 0 0 1...
4,593c3f815635_study,negative 0.001 0 0 1 1 typical 0.710 0 0 1...


In [59]:
if fast_sub:
    final_df = pd.concat([study_df.sort_values(by=['id']), image_df])
    final_df.to_csv('/kaggle/working/submission.csv', index=False)
else:
    df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
    for i in tqdm(range(df.shape[0])):
        level = df.loc[i, 'id'].split('_')[1]
        im_id = df.loc[i, 'id']
        if level == 'study':
            df.loc[i, 'PredictionString'] = study_df[study_df.id==im_id]['PredictionString'].values[0]
        elif level == 'image':
            df.loc[i, 'PredictionString'] = image_df[image_df.id==im_id]['PredictionString'].values[0]
    df.to_csv('/kaggle/working/submission.csv', index=False)

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [60]:
import shutil
shutil.rmtree('/kaggle/tmp/')
shutil.rmtree('/kaggle/working/gdcm')
shutil.rmtree('/kaggle/working/mmdetection')
shutil.rmtree('/kaggle/working/runs')